<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_2D-CNN/blob/main/2D_CNN_AutoEncoder_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#AI_Competition Created by Hyunjun
#Date : 2022.06.02

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalization (0~1)
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
#CNN part
model = models.Sequential()
model.add(layers.Conv2D(20, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2,2), padding='Valid'))
model.add(layers.Conv2D(50, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2,2), padding='Valid'))
model.summary()

In [ ]:
#Dense Layer part
learning_rate = 0.001
batch_size = 128
drop_rate = 0.3
model.add(layers.Flatten())
model.add(layers.Dense(500, use_bias = True, activation = 'relu'))
model.add(layers.Dropout(drop_rate))
model.add(layers.Dense(10, activation = 'softmax'))

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size = batch_size, validation_split=0.2, epochs = 10, verbose = 1)

# Accuracy Visualization
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Loss visualization
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#Testifying
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose = 2)
train_loss, train_acc = model.evaluate(train_images, train_labels, verbose = 2)
print("Test Accuracy : ", test_acc)
print("Train Accuracy : ", train_acc)

In [ ]:
#Noise AutoEncoder with 2D-CNN

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
print(train_images[0].shape)
plt.imshow(train_images[1])
#train_images = train_images.reshape((60000, 28, 28, 1))
#test_images = test_images.reshape((10000, 28, 28, 1))

# Normalization (0~1)
train_images, test_images = train_images / 255.0, test_images / 255.0

x_train = train_images
x_test = test_images
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape) 
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape) 
print(x_train_noisy[0].shape)
x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)
print(x_train_noisy[0].shape)
print(x_test_noisy[0].shape)

In [ ]:
n = 5  
plt.figure(figsize=(n*2, 4))
for i in range(1, n+1):
    # Original Train Data
    ax = plt.subplot(2, n, i)
    plt.imshow(x_train[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
 
    # Noisy Train Data
    ax = plt.subplot(2, n, i + n)
    plt.imshow(x_train_noisy[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
#2D-CNN AutoEncoder Design

learning_rate = 0.001
batch_size = 128

#Encoding
model = models.Sequential()
model.add(layers.Conv2D(50, (3, 3), activation='relu', padding = 'Same', input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2,2), padding = 'Same'))
model.add(layers.Conv2D(25, (3, 3), activation='relu', padding = 'Same'))
model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2,2), padding = 'Same'))

#Decoding
model.add(layers.Conv2D(25, (3, 3), activation='relu',padding = 'Same'))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(50, (3, 3), activation='relu', padding = 'Same'))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='Same'))
model.summary()



In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train_noisy, x_train, batch_size = batch_size, epochs = 10, verbose = 1)
out = model.predict(x_test_noisy)
out = tf.squeeze(out)

n = 5  
plt.figure(figsize=(n*2, 4))
for i in range(1, n+1):
    # Noise Test Data
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test_noisy[i])
    print("Encoded : ",x_test_noisy[i].shape)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
 
    # Noise Reducted Data
    ax = plt.subplot(2, n, i + n)
    plt.imshow(out[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()
